In [ ]:
import os
import cv2
import numpy as np
from random import shuffle
from tqdm import tqdm
#因为图片较多，所以使用tqdm 的模块将处理过程可视化为进度条

In [ ]:
#进行图片预处理

def image_label(image):
    #对图片进行独热标记
  
    label = image.split('.')[-3]
    if label == 'cat':
        return [1, 0]
    elif label == 'dog':
        return [0, 1]

def image_preprocess(dir_path='train'):
   # #图片预处理
  
    data = []
    for img in tqdm(os.listdir(dir_path)): # 调用 tqdm 可视化循环处理过程
        img_path = os.path.join(dir_path, img)
        img_data = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) # 使用 opencv 将图片处理成灰度矩阵
        img_data = cv2.resize(img_data, (64, 64)) # 图片处理成统一大小
        data.append([np.array(img_data), image_label(img)])
    shuffle(data) # 随机打乱
    np.save('data.npy', data) # 存放处理后的 numpy 数组，方便下次使用
    return data

train='E:/tensorflow/train'
data = image_preprocess(train)

In [ ]:
#data = np.load('data.npy',allow_pickle=True)# 加载预处理好的数组文件

In [ ]:
#20000 张划为训练集，另外 5000 张作为测试集。
train_data = data[:-5000] # 训练集
test_data = data[-5000:] # 测试集

In [ ]:
#将训练集的特征数组及标签数组分别分离出来
#进行reshape操作
X_train = np.array([i[0] for i in train_data]).reshape(-1, 64, 64, 1) # 训练集特征
y_train = np.array([i[1] for i in train_data]) # 训练集标签

X_test = np.array([i[0] for i in test_data]).reshape(-1, 64, 64, 1) # 测试集特征
y_test = np.array([i[1] for i in test_data]) # 测试集标签

len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
#构建卷积神经网络模型
#采用卷积层 + 池化层堆叠，最后连接上几个全连接层得到输出的形式


#第一层卷积层，卷积核大小 3\times33×3，包含 32 个卷积核，same padding 形式，relu 激活；
# 第一层池化层，池化大小 2\times22×2， 步长为 1，valid padding 形式；
# 第二层卷积层，卷积核大小 3\times33×3，包含 64 个卷积核，same padding 形式，relu 激活；
# 第二层池化层，池化大小 2\times22×2，步长为 1，valid padding 形式；
# 第三层卷积层，卷积核大小 3\times33×3，包含 128 个卷积核，same padding 形式，relu 激活；
# 第三层池化层，池化大小 2\times22×2，步长为 2，valid padding 形式；
# 全连接层，128 的输出，注意要将第三层池化层平铺送入第一层全连接层；
# Dropout 层，概率设为 0.6；
# 全连接层输出，由于是猫狗的判断输出，则输出大小为 2。

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# 根据上面处理后的图片大小，补充模型输入
x = tf.placeholder(tf.float32,[None,64,64,1])
y = tf.placeholder(tf.int64,[None,2])


# 卷积 + 池化 1
conv1 = tf.layers.conv2d(inputs=x,filters=32,kernel_size=[3,3],padding='same',activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=1)


# 卷积 + 池化 2
conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[3,3],padding='same',activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=1)

# 卷积 + 池化 3
conv3 = tf.layers.conv2d(inputs=pool2,filters=128,kernel_size=[3,3],padding='same',activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(inputs=conv3,pool_size=[2,2],strides=2)

# 全连接层
flatten = tf.layers.flatten(pool3)
dense = tf.layers.dense(inputs=flatten,units=128,activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=dense,rate=0.6)
logits = tf.layers.dense(inputs=dropout,units=2,activation=None) # 输出

In [ ]:
logits

In [ ]:
# 定义损失函数、优化器、以及准确率评估函数。
# 使用交叉熵损失函数。
# 学习率设为 0.001。

# tf.reduce_mean(): 计算平均值,tf.reduce_mean 函数用于计算张量tensor沿着指定的数轴（tensor的某一维度）上的的平均值，主要用作降维或者计算tensor（图像）的平均值。
# tf.argmax(): 返回张量最大值索引。
# tf.cast(): 张量数据类型转换。
# tf.nn.in_top_k(): 判断结果是否在 top k 的预测之中。


# 损失函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y))

# 优化器
training_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

# 准确率计算公式
acc_func = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logits,tf.argmax(y,1),1),tf.float32))


In [ ]:
from matplotlib import pyplot as plt
from IPython import display # 引入 display 模块目的方便程序运行展示
import random
%matplotlib inline

batch_size = 32 # 批量大小

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print('训练开始，请耐心等候')

    for epoch in range(5):
        train_acc_list = [] # 为了绘图
        test_acc_list = [] # 为了绘图
        # 获取批量
        train_batch = zip(range(0, len(X_train), batch_size), range(batch_size, len(X_train) + 1, batch_size))
        i = 1
        for start, end in train_batch:
            sess.run(training_op, feed_dict={x: X_train[start:end], y: y_train[start:end]}) # 传入数据
            train_acc = acc_func.eval(feed_dict={x: X_train[start:end], y: y_train[start:end]}) # 训练集准确率

            test_index = random.sample(range(len(y_test)), batch_size) # 随机抽样一个 batch 大小的测试集
            test_acc = acc_func.eval(feed_dict={x: X_test[test_index], y: y_test[test_index]}) # 测试集准确率
           # 以下是绘图代码
            train_acc_list.append(train_acc)
            test_acc_list.append(test_acc)

            display.clear_output(wait=True)
            plt.style.use('ggplot')
            plt.figure(figsize=(12, 8))
            plt.plot(train_acc_list, label='train_accuracy')
            plt.plot(test_acc_list, label='test_accuracy')
            plt.legend()
            plt.title('epoch: {}, batch:{}, train_accuracy: {}, test_accuracy: {}'.format(epoch+1, i,
                                                                                          train_acc, test_acc))
            plt.show()
            i+=1